# Train model

## Imports

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
import holidays
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.model_selection import cross_validate, StratifiedKFold,cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import make_scorer
from sklearn.metrics import (
        mean_absolute_error,
        mean_squared_error,
        mean_squared_log_error,
        r2_score,
)
from sklearn.preprocessing import RobustScaler, MinMaxScaler
import joblib
import pickle
import tensorflow as tf
import numpy as np
from sklearn.ensemble import RandomForestRegressor

2023-05-16 21:45:02.373880: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# read data
braga_air_data = pd.read_csv('OpenAQ/braga_air_data.csv')

In [3]:
# Replace the AQI column with a new column with the AQI values randomly generated between 0 and 100
braga_air_data['AQI'] = np.random.randint(0, 100, braga_air_data.shape[0])

# export the data to a csv file
braga_air_data.to_csv('OpenAQ/braga_air_data.csv', index=False)

In [4]:
# head
braga_air_data.head()

,latitude,longitude,AQI,year,month,day,hour,weekday,season,part_of_day,is_weekend,is_holiday
0,41.549722,-8.405833,83,2017,12,29,4,4,1,4,0,0
1,41.549722,-8.405833,48,2017,12,29,4,4,1,4,0,0
2,41.549722,-8.405833,11,2017,12,29,3,4,1,4,0,0
3,41.549722,-8.405833,63,2017,12,29,3,4,1,4,0,0
4,41.549722,-8.405833,26,2017,12,29,2,4,1,4,0,0


In [5]:
# drop is weekend column and is holiday column
braga_air_data = braga_air_data.drop(['is_weekend', 'is_holiday'], axis=1)

# normalize latitude and longitude
braga_air_data['latitude'] = (braga_air_data['latitude'] - braga_air_data['latitude'].min()) / (
            braga_air_data['latitude'].max() - braga_air_data['latitude'].min())
braga_air_data['longitude'] = (braga_air_data['longitude'] - braga_air_data['longitude'].min()) / (
            braga_air_data['longitude'].max() - braga_air_data['longitude'].min())
# export the data to a csv file
braga_air_data.to_csv('OpenAQ/braga_air_data.csv', index=False)

In [7]:
#divisão do target
x = braga_air_data.drop(['AQI'], axis=1)
y = braga_air_data['AQI']

In [8]:
# metrics
METRICS = {
    "MAE": make_scorer(mean_absolute_error),
    "MSE": make_scorer(mean_squared_error,squared=True),
    "RMSE": make_scorer(mean_squared_error,squared=False),
    "r2_Score": make_scorer(r2_score),
    "RMSLE": make_scorer(mean_squared_log_error,squared=False)
}

In [9]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=1234)
#Função de kfold com 10 iterações, garante que os resultados são iguais

rf = RandomForestRegressor(max_depth=3, random_state=1234)
scores = cross_validate(rf, x, y, cv=splitter, scoring=METRICS)
rf_scores = pd.DataFrame(scores)
pd.DataFrame(rf_scores.mean()).T

,fit_time,score_time,test_MAE,test_MSE,test_RMSE,test_r2_Score,test_RMSLE
0,1.178857,0.012665,0.0,0.0,0.0,1.0,0.0


In [10]:
Ridge = Ridge()
scores = cross_validate(Ridge, x, y, cv=splitter, scoring=METRICS)
Ridge_scores = pd.DataFrame(scores)
pd.DataFrame(Ridge_scores.mean()).T

,fit_time,score_time,test_MAE,test_MSE,test_RMSE,test_r2_Score,test_RMSLE
0,0.013529,0.002978,0.690924,9.635436,3.103922,0.965521,NaN


In [11]:
ls = Lasso()
scores = cross_validate(ls, x, y, cv=splitter, scoring=METRICS)
ls_scores = pd.DataFrame(scores)
pd.DataFrame(ls_scores.mean()).T

,fit_time,score_time,test_MAE,test_MSE,test_RMSE,test_r2_Score,test_RMSLE
0,0.011557,0.002778,7.280774,90.074979,9.490662,0.677676,NaN


In [12]:
gb = GradientBoostingRegressor(random_state=1234)
scores = cross_validate(gb, x, y, cv=splitter, scoring=METRICS)
gb_scores = pd.DataFrame(scores)
pd.DataFrame(gb_scores.mean()).T

,fit_time,score_time,test_MAE,test_MSE,test_RMSE,test_r2_Score,test_RMSLE
0,2.062684,0.00713,0.000441,1.971575e-07,0.000444,1.0,0.000307


In [13]:
lr = LinearRegression()
scores = cross_validate(lr, x, y, cv=splitter, scoring=METRICS)
lr_scores = pd.DataFrame(scores)
pd.DataFrame(lr_scores.mean()).T

,fit_time,score_time,test_MAE,test_MSE,test_RMSE,test_r2_Score,test_RMSLE
0,0.035646,0.004528,1.187329e-11,2.478088e-22,1.476238e-11,1.0,1.233740e-11


In [14]:
model = 'rf'
# Salvar o modelo em disco
joblib.dump(rf, f"{model}_model.pkl")


['rf_model.pkl']